In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer/sample_submission.csv
/kaggle/input/customer/Train.csv
/kaggle/input/customer/Test.csv


In [2]:
df_data = pd.read_csv("../input/customer/Train.csv")
df_data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [3]:
df_data = df_data.rename(columns ={'Var_1': 'Category'})
df_data.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [4]:
df_data = df_data.drop(columns=['ID'])
df_data.sample(10)             

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
6799,Female,No,28,Yes,Healthcare,1.0,Low,6.0,Cat_6,D
7438,Male,Yes,28,No,Doctor,5.0,Low,2.0,Cat_3,D
5283,Female,Yes,56,No,Artist,NaN,Average,3.0,Cat_6,C
5804,Male,Yes,35,No,Artist,9.0,Low,3.0,Cat_7,A
6744,Female,No,63,Yes,Artist,1.0,Low,2.0,Cat_6,B
5510,Male,Yes,53,Yes,Artist,4.0,Average,2.0,Cat_6,C
6890,Female,Yes,66,Yes,Artist,1.0,Average,3.0,Cat_6,C
8055,Male,No,18,No,Healthcare,0.0,Low,2.0,Cat_6,D
1856,Female,Yes,40,Yes,Artist,1.0,Average,2.0,Cat_6,C
3081,Male,Yes,52,No,Executive,9.0,Average,4.0,Cat_6,A


In [5]:
df_data.isna().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Category            76
Segmentation         0
dtype: int64

In [6]:
df_data.dropna().sum()
df_data = df_data.dropna()
df_data.sample(10)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
1680,Male,Yes,65,Yes,Executive,0.0,High,4.0,Cat_6,C
8013,Male,Yes,52,Yes,Artist,2.0,Low,3.0,Cat_3,A
3087,Female,No,30,Yes,Doctor,0.0,Low,5.0,Cat_6,D
7496,Male,Yes,40,Yes,Entertainment,8.0,Low,2.0,Cat_6,D
7635,Female,Yes,57,Yes,Artist,1.0,High,3.0,Cat_2,B
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1556,Male,Yes,36,No,Entertainment,0.0,Average,4.0,Cat_4,A
3140,Female,No,42,Yes,Artist,9.0,Low,1.0,Cat_6,C
5772,Male,Yes,47,Yes,Executive,8.0,High,4.0,Cat_6,C
3620,Female,Yes,26,No,Homemaker,8.0,Average,4.0,Cat_2,D


# Covert non-numerical values into numerical vaues

In [7]:
def get_encoded_dict(values):
    return {val:i for i, val in enumerate(values)}

In [8]:
gender_dict = get_encoded_dict(df_data.Gender.unique())
married_dict = get_encoded_dict(df_data.Ever_Married.unique())
grad_dict = get_encoded_dict(df_data.Graduated.unique())
Profession_dict = get_encoded_dict(df_data.Profession.unique())
spend_dict = get_encoded_dict(df_data.Spending_Score.unique())
cat_dict = get_encoded_dict(df_data.Category.unique())
segment_dict = get_encoded_dict(df_data.Segmentation.unique())
segment_dict

{'D': 0, 'B': 1, 'C': 2, 'A': 3}

In [9]:
encoded_dict = {
    'Gender' : gender_dict,
    'Ever_Married': married_dict,
    'Graduated' : grad_dict,
    'Profession': Profession_dict,
    'Spending_Score': spend_dict,
     'Category': cat_dict,
    'Segmentation': segment_dict
}

In [10]:
df_data = df_data.replace(encoded_dict)
df_data.sample(10)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
4382,0,1,39,0,8,12.0,1,7.0,1,1
3123,0,1,38,1,6,0.0,0,2.0,0,3
2017,0,0,30,1,4,0.0,0,2.0,3,3
3634,0,0,42,1,6,0.0,0,3.0,1,3
2171,0,0,29,1,3,4.0,0,1.0,3,0
1965,0,1,43,1,3,2.0,0,2.0,1,1
6823,0,0,32,0,4,2.0,0,2.0,1,0
5032,0,1,42,1,6,1.0,0,2.0,1,0
3703,0,1,60,1,3,3.0,2,2.0,1,2
5722,0,1,60,1,3,0.0,0,1.0,1,1


In [11]:
df_data.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Category',
       'Segmentation'],
      dtype='object')

In [12]:
feature_cols = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession','Work_Experience', 'Spending_Score', 'Family_Size', 'Category']
target_col = ['Segmentation']

In [13]:
from sklearn.model_selection import train_test_split
data_x = df_data.loc[:, feature_cols]
data_y = df_data.loc[:, target_col]
train_x,test_x,train_y,test_y = train_test_split(data_x, data_y, test_size=0.2)

In [14]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(5332, 9)
(5332, 1)
(1333, 9)
(1333, 1)


In [15]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB()
cnb.fit(train_x.to_numpy(), train_y.to_numpy().reshape(-1))

CategoricalNB()

In [16]:
actual_y = test_y.to_numpy().reshape(-1)
predicted_y = cnb.predict(test_x.to_numpy())

In [17]:
actual_y

array([1, 1, 1, ..., 0, 2, 0])

In [18]:
predicted_y

array([2, 3, 2, ..., 0, 0, 3])

In [19]:
from sklearn import metrics
metrics.accuracy_score(actual_y, predicted_y)

0.4981245311327832

# Confusion matrix,Classification_Report

In [20]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(actual_y,predicted_y))

[[222  18  14  70]
 [ 30  69 147  84]
 [ 49  48 215  32]
 [ 54  50  73 158]]


In [21]:
segment_dict

{'D': 0, 'B': 1, 'C': 2, 'A': 3}

In [22]:
print(classification_report(actual_y, predicted_y, target_names=['D','B','C','A']))

              precision    recall  f1-score   support

           D       0.63      0.69      0.65       324
           B       0.37      0.21      0.27       330
           C       0.48      0.62      0.54       344
           A       0.46      0.47      0.47       335

    accuracy                           0.50      1333
   macro avg       0.48      0.50      0.48      1333
weighted avg       0.48      0.50      0.48      1333

